# README — Модель R(t) и прогноз по COVID-19 (Россия, Италия, Германия, Франция)

## 1) Постановка задачи

В рамках задания нужно:

1. **Прокомментировать код** — объяснить, зачем используется каждая ключевая строка/блок.  
2. **Обучить модель** на данных о новых выявленных случаях COVID-19 за период **01.01.2020—01.12.2020** по странам: **Россия, Италия, Германия, Франция**.  
   При этом **игнорировать все дни до того, как дневная заболеваемость превысит 100 случаев**.  
3. **Оценить динамику эффективного репродуктивного числа R(t)** на этом периоде.  
4. **Сделать прогноз** числа зарегистрированных случаев **и** R(t) на даты **02.12.2020—14.12.2020** и **сравнить с реальностью**.

---

## 2) Порядок решения

- **Данные.** Загружаем открытые CSV из OWID: данные по дням (JHU) и тестирование (OWID testing).  
- **Фильтр.** Для каждой страны находим **первый день с `new_cases > 100`**, отсекаем всё ранее. Учебное окно — до **2020-12-01** включительно.  
- **Модель инфекций.** Не **SIR**, а **renewal-модель** (уравнение обновления): инфекции на день *t* зависят от прошлых инфекций, свернутых с **generation interval**, и масштабируются **R(t)**.  
- **Задержки наблюдения.** Моделируем **инкубацию + задержку подтверждения** и сворачиваем инфекции с распределением задержки, получая ожидаемые «подтвержденные» случаи.  
- **Тестовая активность.** Вводим **exposure** — прокси «интенсивности тестирования», чтобы не переоценивать случаи, когда тестов мало.  
- **Наблюдение.** Реальные «положительные» — счётчики с **пере-дисперсией**, поэтому используем **Negative Binomial**.  
- **R(t).** Ставим **гауссовский random walk по `log R(t)`**, изменяющийся раз в 3 дня.  
- **Вывод.** Байесовский вывод через **NUTS** (NumPyro/JAX или PyMC).  
- **Прогноз.** Делаем **posterior predictive** на будущее окно (02.12—14.12.2020) и сравниваем с фактами.

---

## 3) Математика модели

### 3.1 Renewal-динамика инфекций

Пусть $y_t$ — число **новых инфекций** (истинных, до задержек) в день $t$.  
Пусть $g_k$ — дискретизированное **generation interval** (вероятность, что вторичная инфекция произойдёт через $k$ дней).  
Тогда уравнение обновления:

$$
y_t = R_t \sum_{k=1}^{K} y_{t-k} g_k, \qquad y_0 \sim \text{Exponential}(\lambda)
$$

- $R_t$ — эффективное репродуктивное число на день $t$.  
- $g_k$ берётся из логнормального приближения и нормируется так, чтобы $\sum_k g_k = 1$.

### 3.2 Приор для $R_t$

Работаем с $\log R_t$ и задаём гауссовский случайный блуждающий процесс:

$$
\log R_{t}^{(\text{coarse})} \sim \text{GRW}(\sigma), \quad
\log R_t = \text{repeat}(\log R_{t}^{(\text{coarse})},\ \text{step}=3)
$$

То есть $R_t$ меняется раз в 3 дня — траектория гладкая, но гибкая.  
Далее $R_t = \exp(\log R_t)$.

### 3.3 Задержки (инкубация + от симптомов до подтверждения)

Пусть $d_\tau$ — распределение задержки от инфекции до регистрации.  
Инкубация ≈ 5 дней, плюс логнормальная задержка (mean 4.7, std 2.9).  

Ожидаемые регистрируемые (до тестового масштаба) случаи:

$$
c_t^{(\text{pre-test})} = \sum_{\tau=0}^{D} y_{t-\tau} d_\tau
$$

### 3.4 Масштаб по тестам (exposure)

Чтобы учесть ограниченность тестирования, вводим нижний порог baseline и определяем:

$$
\text{exposure}_t = \max(\text{tests}_t,\ 0.1 \times \max(\text{tests}))
$$
$$
\mu_t = \text{exposure}_t \cdot c_t^{(\text{pre-test})}
$$

Здесь $\mu_t$ — ожидаемое число подтверждённых случаев в день $t$.

### 3.5 Наблюдательная модель (Negative Binomial)

Реальные дневные случаи шумные и пере-дисперсные:

$$
C_t \sim \text{NegBinom}(\mu_t,\ \alpha), \quad
\alpha \sim \text{Gamma}(\mu=4,\ \sigma=2)
$$

Мы подбираем $C_t$ только для дней, где тестов больше 0.

### 3.6 Постериорный вывод и прогноз

- Оцениваем совместное апостериорное распределение $\{R_t, y_t, \alpha, \text{seed}\}$.  
- Для прогноза продолжаем динамику $y_t$, вычисляем $\mu_t$, генерируем $C_t$.  
- Для R(t) берём медиану и 80% HDI.

---

# 4) Монте-Карло / Байесовский вывод (NUTS) — логический обзор

В проекте используется **байесовский метод Монте-Карло (MCMC)** с адаптивным самплером **NUTS (No-U-Turn Sampler)**, встроенным в PyMC. Этот алгоритм позволяет получить апостериорные распределения параметров модели ($R_t$, количество инфекций, дисперсию и др.), когда аналитическое решение невозможно.

## 4.1) Алгоритм MCMC, используемый в проекте (NUTS)

В нашем проекте используется не классический **Metropolis–Hastings**, а более современный алгоритм **NUTS (No-U-Turn Sampler)** — адаптивная версия **Hamiltonian Monte Carlo (HMC)**. Этот метод реализован в PyMC и NumPyro и является стандартом для байесовского моделирования. Ниже — логическое объяснение различий и сути алгоритма.

---

### 🧩 1. Классический Metropolis–Hastings

Метод MH строит цепь случайных выборок:
1. Из текущего состояния параметров $\theta_t$ предлагается новое значение $\theta'$ из распределения предложений $q(\theta'|\theta_t)$.
2. Вычисляется вероятность принятия:
   $$
   A = \min\!\left(1, \frac{p(\theta'|D)\, q(\theta_t|\theta')}{p(\theta_t|D)\, q(\theta'|\theta_t)}\right)
   $$
3. Кандидат $\theta'$ принимается с вероятностью $A$.

MH прост, но плохо масштабируется в многомерных пространствах: шаги случайны, и цепь двигается медленно (эффект случайного блуждания).

---

### ⚙️ 2. Hamiltonian Monte Carlo (HMC)

HMC ускоряет исследование пространства параметров, используя **градиенты логарифма апостериорного распределения**.  
Для параметров $\theta$ вводится вспомогательный импульс $r$, и система описывается *гамильтонианом*:
$$
H(\theta, r) = -\log p(\theta|D) + \frac{1}{2}r^T M^{-1}r
$$
Алгоритм численно интегрирует уравнения движения, получая траекторию, вдоль которой вероятность остаётся примерно постоянной.  
Это позволяет делать длинные «информированные» шаги, а не случайные прыжки, как в MH.

---

### 🧭 3. NUTS — No-U-Turn Sampler

Алгоритм **NUTS** улучшает HMC: он автоматически решает, **насколько далеко** продвигаться вдоль траектории и **какой шаг интегрирования** использовать.  
Когда траектория начинает «поворачивать назад» (делать U-образный разворот), шаг прекращается — отсюда и название *No-U-Turn*.

**Преимущества NUTS:**
- не требует ручной настройки длины пути;
- адаптивно подбирает шаг и матрицу масс;
- сохраняет эффективность HMC, избегая случайных блужданий.

---

## 1. Подготовка данных
Функция `process_country` загружает временные ряды `new_cases` и `new_tests` из OWID, оставляет только даты после того, как `new_cases > 100`, и до `2020-12-01`.  
Полученные значения помещаются в таблицу `observed` со столбцами:
- `positive` — число новых подтверждённых случаев,
- `total` — количество проведённых тестов.  
Эти данные передаются в класс `GenerativeModel`, где строится стохастическая модель.

## 2. Вероятностная структура модели
Метод `GenerativeModel.build()` задаёт основные стохастические зависимости:
- **Приор на динамику $R_t$** — случайное блуждание логарифма:
  ```python
  log_r_coarse = pm.GaussianRandomWalk("log_r_coarse", sigma=0.035)
  r_t = pm.Deterministic("r_t", pt.exp(pt.repeat(log_r_coarse, step)))
  ```
  Это задаёт плавную траекторию $R_t$, обновляющуюся каждые 3 дня и ограничивающую резкие колебания.  
- **Renewal-модель инфекций** — описывает распространение заражений:
  $$
  y_t = R_t \sum_{k=1}^{K} y_{t-k} g_k,
  $$
  где $g_k$ — распределение интервала генерации. В коде вычисляется через `pytensor.scan`.  
- **Учёт задержек регистрации** — свёртка инфекций с распределением задержек (инкубация + время до подтверждения) при помощи `conv2d`.  
- **Наблюдательная модель** — реальные подтверждённые случаи подчиняются распределению Negative Binomial:
  $$
  C_t \sim \text{NegBinom}(\mu_t,\alpha),
  $$
  где $\mu_t$ — ожидаемое число наблюдаемых случаев, $\alpha$ — параметр пере-дисперсии.

## 3. Семплирование апостериорных распределений
После построения модели выполняется MCMC-семплирование с помощью NUTS:
```python
sample_numpyro_nuts(...)  # если доступен JAX
# или
pm.sample(...)             # стандартный PyMC
```
Алгоритм NUTS адаптивно выбирает длину траектории и шаг интеграции, эффективно исследуя пространство параметров. В результате формируется множество выборок из апостериорного распределения (обычно несколько тысяч для каждой цепи).

## 4. Постобработка и прогноз
Из апостериорных выборок вычисляются прогнозные значения $C_t$ и $R_t$ на будущие даты (02.12–14.12.2020).  
На их основе строятся средние прогнозы, доверительные интервалы и сравнение с реальными данными.

---

## 5) Итог

- Загружает данные и тесты для выбранных стран.  
- Отсекаем ранний период до 100 случаев/день.  
- Оценивает $R(t)$ через байесовскую renewal-модель.  
- Даёт прогноз случаев и $R(t)$ на 02.12—14.12.2020.  
- Сравнивает прогноз с реальными данными.  
